###Atleast A100 GPU is required for this

In [2]:
!pip install qwen-vl-utils yt-dlp flash-attn --no-build-isolation accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 25.1 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.7.2.post1-cp310-cp310-linux_x86_64.whl size=190160474 sha256=0b454d9e650bfc437cc71335080172a5d05f51eab355636c9d5b7321fec7318e
  Stored in directory: /root/.cache/pip/wheels/da/ec/5b/b2c37a8e4f755ad82492a822463bca0817f0e0e11de874b550
Successfully built flash-attn


In [4]:
from yt_dlp import YoutubeDL

YOUTUBE_URL = "https://www.youtube.com/watch?v=MCWJNOfJoSM"

ydl_opts = {
    "format": "best",
    "outtmpl":"../data/youtube.mp4"
    }
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([YOUTUBE_URL])

[youtube] Extracting URL: https://www.youtube.com/watch?v=MCWJNOfJoSM
[youtube] MCWJNOfJoSM: Downloading webpage
[youtube] MCWJNOfJoSM: Downloading ios player API JSON
[youtube] MCWJNOfJoSM: Downloading mweb player API JSON
[youtube] MCWJNOfJoSM: Downloading player 3ede36f2
[youtube] MCWJNOfJoSM: Downloading m3u8 information
[info] MCWJNOfJoSM: Downloading 1 format(s): 18
[download] Destination: ../data/youtube.mp4
[download] 100% of   39.07MiB in 00:00:01 at 35.36MiB/s  


In [6]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

MODEL_NAME = "Qwen/Qwen2-VL-7B-Instruct"

model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    attn_implementation="flash_attention_2",
)

processor = AutoProcessor.from_pretrained(MODEL_NAME)

model-00002-of-00005.safetensors:  44%|####4     | 1.71G/3.86G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [1]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "../data/youtube.mp4",
                "max_pixels": 1280 * 780,
                "fps": 0.1,
            },
            {"type": "text", "text": "What's happening in the video? who wins the penalty shootout?."},
        ],
    }
]

text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

image_inputs, video_inputs = process_vision_info(messages)

inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

generated_ids = model.generate(**inputs, max_new_tokens=512)

generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

print(output_text)

NameError: name 'processor' is not defined